In [31]:
import pandas as pd
import numpy as np
import string
import joblib

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.utils import resample
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

from scipy.sparse import hstack

In [32]:
def clean_text(text):
    if not isinstance(text, str):
        return ""
    return text.strip()


In [33]:
emoji_set = set("😀😁😂🤣😃😄😅😆😉😊😋😎😍😘😗😙😚🙂🤗🤩🤔🤨😐😑😶🙄😏😣😥😮🤐😯😪😫😴😌🤓😛😜😝🤤😒😓😔😕🙃🤑😲☹️🙁😖😞😟😤😢😭😦😧😨😩🤯😬😰😱😳🤪😵😡😠🤬")

class ExtraFeatures(BaseEstimator, TransformerMixin):
    def fit(self, x, y=None):
        return self

    def transform(self, texts):
        emoji_count = np.array([sum(1 for ch in t if ch in emoji_set) for t in texts]).reshape(-1, 1)
        punctuation_ratio = np.array([
            sum(1 for c in t if c in string.punctuation) / (len(t) + 1)
            for t in texts
        ]).reshape(-1, 1)
        digit_ratio = np.array([
            sum(1 for c in t if c.isdigit()) / (len(t) + 1)
            for t in texts
        ]).reshape(-1, 1)
        avg_word_len = np.array([
            np.mean([len(w) for w in t.split()]) if len(t.split()) > 0 else 0
            for t in texts
        ]).reshape(-1, 1)

        return np.hstack([emoji_count, punctuation_ratio, digit_ratio, avg_word_len])


In [43]:
df = pd.read_csv("training_data.csv")

human_df = pd.DataFrame({"text": df["Human_Content"], "label": "human"})
ai_df = pd.DataFrame({"text": df["AI_Content"], "label": "ai"})  # Dummy AI row (replace later)

df_final = pd.concat([human_df, ai_df], ignore_index=True)
df_final = df_final.dropna()

df_final["clean_text"] = df_final["text"].apply(clean_text)


In [44]:
human = df_final[df_final.label == "human"]
ai = df_final[df_final.label == "ai"]

if len(human) > len(ai):
    ai = resample(ai, replace=True, n_samples=len(human), random_state=42)
else:
    human = resample(human, replace=True, n_samples=len(ai), random_state=42)

df_balanced = pd.concat([human, ai]).sample(frac=1, random_state=42)


In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    df_balanced["clean_text"], df_balanced["label"],
    test_size=0.10, random_state=42, stratify=df_balanced["label"]
)


In [46]:
tfidf = TfidfVectorizer(max_features=15000, ngram_range=(1, 3), sublinear_tf=True)

X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

extra = ExtraFeatures()
X_train_extra = extra.fit_transform(X_train)
X_test_extra = extra.transform(X_test)

X_train_combined = hstack([X_train_tfidf, X_train_extra])
X_test_combined = hstack([X_test_tfidf, X_test_extra])


In [47]:
model = LinearSVC()
model.fit(X_train_combined, y_train)

print("Model trained successfully!")


Model trained successfully!


In [48]:
y_pred = model.predict(X_test_combined)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.9838709677419355

Classification Report:
               precision    recall  f1-score   support

          ai       0.97      1.00      0.98        31
       human       1.00      0.97      0.98        31

    accuracy                           0.98        62
   macro avg       0.98      0.98      0.98        62
weighted avg       0.98      0.98      0.98        62


Confusion Matrix:
 [[31  0]
 [ 1 30]]


In [49]:
joblib.dump(model, "ai_human_model.pkl")
joblib.dump(tfidf, "vectorizer.pkl")
joblib.dump(extra, "extra_features.pkl")

print("Model & vectorizers saved successfully!")


Model & vectorizers saved successfully!


In [50]:
def predict_text(text):
    vec = tfidf.transform([text])
    ext = extra.transform([text])
    combined = hstack([vec, ext])
    return model.predict(combined)[0]


In [51]:
tests = [
    "This research demonstrates a novel AI framework for text generation.",
    "Bro I just woke up and I'm still sleepy 😂",
    "The system architecture uses transformer layers for encoding.",
    "I went to the market and bought snacks.",
    "AI improves efficiency in multiple domains."
]

for t in tests:
    print("\nText:", t)
    print("Prediction:", predict_text(t))



Text: This research demonstrates a novel AI framework for text generation.
Prediction: ai

Text: Bro I just woke up and I'm still sleepy 😂
Prediction: human

Text: The system architecture uses transformer layers for encoding.
Prediction: ai

Text: I went to the market and bought snacks.
Prediction: ai

Text: AI improves efficiency in multiple domains.
Prediction: ai
